# 01 Import libraries

In [1]:
import numpy as np
import torch

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using device: {device}")

Using device: cpu


In [2]:
import pandas as pd

In [3]:
import importlib

## Import our own modules

In [4]:
!rm -r Learning-the-Optimal-Solution-Path
!git clone https://github.com/Cumberkid/Learning-the-Optimal-Solution-Path.git

rm: cannot remove 'Learning-the-Optimal-Solution-Path': No such file or directory
Cloning into 'Learning-the-Optimal-Solution-Path'...
remote: Enumerating objects: 1744, done.
remote: Counting objects: 100% (714/714), done.
remote: Compressing objects: 100% (355/355), done.
remote: Total 1744 (delta 542), reused 479 (delta 359), pack-reused 1030 (from 1)
Receiving objects: 100% (1744/1744), 13.25 MiB | 16.37 MiB/s, done.
Resolving deltas: 100% (1179/1179), done.


(Using Colab)

In [5]:
import sys

In [6]:
# Add the parent directory to sys.path
sys.path.append('/content/Learning-the-Optimal-Solution-Path')

In [7]:
import lib
importlib.reload(lib)

<module 'lib' from '/content/Learning-the-Optimal-Solution-Path/lib/__init__.py'>

In [8]:
from lib.lsp.basis_generator import scaled_shifted_legendre
from lib.lsp.basis_tf_module import Basis_TF_SGD
from lib.lsp.learn_solution_path import lsp_boosting
from lib.lsp.loss_fn_lsp import reg_unif_weighted_logit
from lib.fast_tensor_data_loader import FastTensorDataLoader

# 02 Load data

In [9]:
# file path for Colab. May need to change this
X_df = pd.read_csv('/content/Learning-the-Optimal-Solution-Path/experiments/fair-regression/data/X_processed.csv')
y_df = pd.read_csv('/content/Learning-the-Optimal-Solution-Path/experiments/fair-regression/data/y_processed.csv')

In [10]:
X = np.array(X_df)
y = np.array(y_df).squeeze()

In [11]:
train_X = torch.tensor(X, dtype=torch.float32)
train_y = torch.tensor(y, dtype=torch.float32)

In [12]:
# full gradient descent uses all data points
GD_data_loader = FastTensorDataLoader(train_X, train_y, batch_size=1000, shuffle=True, )
# stochastic gradient descent uses mini-batch
SGD_data_loader = FastTensorDataLoader(train_X, train_y, batch_size=20, shuffle=True, )
# test data
test_data_loader = FastTensorDataLoader(train_X, train_y, batch_size=1000, shuffle=False, )

In [13]:
lam_max = 1
lam_min = 0
input_dim = X.shape[1]
loss_fn = reg_unif_weighted_logit

In [14]:
# Read the CSV file into a DataFrame
# truth = pd.read_csv('/content/Learning-the-Optimal-Solution-Path/experiments/fair-regression/results/exact_soln_list.csv')
truth = pd.read_csv('exact_soln_list_legendre.csv')

# Display the DataFrame
truth

,losses,theta_0,theta_1,theta_2,theta_3,theta_4,theta_5,theta_6,theta_7,theta_8,...,theta_36,theta_37,theta_38,theta_39,theta_40,theta_41,theta_42,theta_43,theta_44,theta_45
0,0.190793,-0.373029,0.186622,0.227443,-0.023583,0.164822,0.142992,0.168018,0.026305,0.142661,...,-0.018010,0.054908,-0.005629,0.000000,0.000000,-0.031270,-0.018010,-0.139608,-0.116840,-0.061673
1,0.192130,-0.371212,0.186634,0.227586,-0.023679,0.164875,0.142262,0.167428,0.025960,0.142993,...,-0.017600,0.054199,-0.005657,0.000069,0.000098,-0.031115,-0.017600,-0.139385,-0.116402,-0.061226
2,0.193463,-0.369401,0.186645,0.227729,-0.023775,0.164927,0.141535,0.166842,0.025615,0.143323,...,-0.017191,0.053493,-0.005685,0.000137,0.000195,-0.030961,-0.017191,-0.139162,-0.115965,-0.060781
3,0.194792,-0.367595,0.186656,0.227870,-0.023870,0.164977,0.140811,0.166258,0.025271,0.143653,...,-0.016784,0.052790,-0.005713,0.000205,0.000293,-0.030808,-0.016784,-0.138939,-0.115529,-0.060337
4,0.196117,-0.365794,0.186665,0.228009,-0.023964,0.165027,0.140089,0.165677,0.024927,0.143982,...,-0.016378,0.052089,-0.005741,0.000273,0.000390,-0.030655,-0.016378,-0.138717,-0.115094,-0.059894
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1019,0.327684,0.692472,-0.029101,-0.033382,-0.017462,-0.005831,-0.225541,-0.146453,-0.051680,0.070009,...,0.105324,-0.201982,0.000913,0.013100,0.018598,0.063129,0.105324,0.157442,0.161747,0.171301
1020,0.325866,0.693930,-0.029932,-0.034521,-0.017548,-0.006565,-0.226194,-0.148351,-0.051246,0.068693,...,0.105340,-0.202186,0.000990,0.013040,0.018537,0.063366,0.105340,0.158131,0.162149,0.171464
1021,0.324032,0.695399,-0.030769,-0.035668,-0.017634,-0.007304,-0.226853,-0.150274,-0.050809,0.067366,...,0.105357,-0.202393,0.001066,0.012980,0.018475,0.063604,0.105357,0.158825,0.162554,0.171627
1022,0.322184,0.696878,-0.031613,-0.036822,-0.017720,-0.008050,-0.227517,-0.152223,-0.050369,0.066028,...,0.105374,-0.202601,0.001143,0.012919,0.018415,0.063843,0.105374,0.159523,0.162962,0.171792


In [15]:
selected_columns = ['theta_0', 'theta_1', 'theta_2', 'theta_3', 'theta_4',
                    'theta_5', 'theta_6', 'theta_7', 'theta_8', 'theta_9',
                    'theta_10', 'theta_11', 'theta_12', 'theta_13', 'theta_14',
                    'theta_15', 'theta_16', 'theta_17', 'theta_18', 'theta_19',
                    'theta_20', 'theta_21', 'theta_22', 'theta_23', 'theta_24',
                    'theta_25', 'theta_26', 'theta_27', 'theta_28', 'theta_29',
                    'theta_30', 'theta_31', 'theta_32', 'theta_33', 'theta_34',
                    'theta_35', 'theta_36', 'theta_37', 'theta_38', 'theta_39',
                    'theta_40', 'theta_41', 'theta_42', 'theta_43', 'theta_44',
                    'theta_45']
true_thetas = truth[selected_columns].to_numpy()
true_losses = truth['losses'].to_numpy()

In [16]:
# import time

# 03 Exact gradient

In [17]:
max_epochs = 300

In [18]:
phi_lam = scaled_shifted_legendre

In [19]:
start_basis_dim = 5
end_basis_dim = 13
init_lr = 0.125

In [ ]:
def thresh_basis(basis_dim):
    return 0.1**(basis_dim/5 + 2.5)

In [ ]:
np.random.seed(8675309)
torch.manual_seed(8675309)

num_itr_history, sup_err_history, breaks = lsp_boosting(input_dim, start_basis_dim, end_basis_dim,
                                        phi_lam, max_epochs, GD_data_loader,  test_data_loader,
                                        loss_fn, lam_min, lam_max, true_losses, init_lr=init_lr,
                                        thresh_basis=thresh_basis, record_frequency=10,
                                        distribution='uniform', device=device, trace_frequency=10)

********** now running lsp with #basis dimension = 5 ***********
--------approximate solution path for # itr = 10 complete--------
# epoch: 10	 sup error: 3.808290511369705
--------approximate solution path for # itr = 20 complete--------
# epoch: 20	 sup error: 0.4882354736328125
--------approximate solution path for # itr = 30 complete--------
# epoch: 30	 sup error: 0.3149908185005188
--------approximate solution path for # itr = 40 complete--------
# epoch: 40	 sup error: 0.06118583679199219
--------approximate solution path for # itr = 50 complete--------
# epoch: 50	 sup error: 0.040966540575027466
--------approximate solution path for # itr = 60 complete--------
# epoch: 60	 sup error: 0.02363187074661255
--------approximate solution path for # itr = 70 complete--------
# epoch: 70	 sup error: 0.01431553065776825
--------approximate solution path for # itr = 80 complete--------
# epoch: 80	 sup error: 0.009075149893760681
--------approximate solution path for # itr = 90 complete

In [20]:
def thresh_basis(basis_dim):
    return 0.1**(4)

In [21]:
np.random.seed(8675309)
torch.manual_seed(8675309)

num_itr_history, sup_err_history, breaks = lsp_boosting(input_dim, start_basis_dim, end_basis_dim,
                                        phi_lam, max_epochs, GD_data_loader,  test_data_loader,
                                        loss_fn, lam_min, lam_max, true_losses, init_lr=init_lr,
                                        thresh_basis=thresh_basis, record_frequency=10,
                                        distribution='uniform', device=device, trace_frequency=10)

********** now running lsp with #basis dimension = 5 ***********
--------approximate solution path for # itr = 10 complete--------
# epoch: 10	 sup error: 3.808290511369705
--------approximate solution path for # itr = 20 complete--------
# epoch: 20	 sup error: 0.4882354736328125
--------approximate solution path for # itr = 30 complete--------
# epoch: 30	 sup error: 0.314990758895874
--------approximate solution path for # itr = 40 complete--------
# epoch: 40	 sup error: 0.061185866594314575
--------approximate solution path for # itr = 50 complete--------
# epoch: 50	 sup error: 0.04096655547618866
--------approximate solution path for # itr = 60 complete--------
# epoch: 60	 sup error: 0.023631885647773743
--------approximate solution path for # itr = 70 complete--------
# epoch: 70	 sup error: 0.01431553065776825
--------approximate solution path for # itr = 80 complete--------
# epoch: 80	 sup error: 0.009075149893760681
--------approximate solution path for # itr = 90 complete

In [ ]:
pd.DataFrame(np.array(breaks), columns=['breaks']).to_csv('LSP_results_exact_boosted_breaks_legendre.csv', index=False)

In [ ]:
file_path = 'LSP_results_exact_boosted_legendre.csv'

LSP_results_exact = pd.DataFrame(np.column_stack((num_itr_history, sup_err_history)), columns=['num_itr', 'sup_err'])

# Save the DataFrame to a CSV file
LSP_results_exact.to_csv(file_path, index=False)

# Read the CSV file into a DataFrame
df = pd.read_csv(file_path)

# Display the DataFrame
df

,num_itr,sup_err
0,10.0,3.808291e+00
1,20.0,4.882355e-01
2,30.0,3.149908e-01
3,40.0,6.118584e-02
4,50.0,4.096654e-02
5,60.0,2.363187e-02
6,70.0,1.431553e-02
7,80.0,9.075150e-03
8,90.0,6.006986e-03
9,100.0,4.100710e-03


# Noisy gradient oracle

In [ ]:
max_epochs = 200

In [ ]:
phi_lam = scaled_shifted_legendre

In [ ]:
def thresh_basis(basis_dim):
    return 0.1**(basis_dim/5 + 5) * (50**2)

In [ ]:
# beta/sqrt(t)
def step_size(t, beta):
    return beta/np.sqrt(t+1)

In [ ]:
start_basis_dim = 5
end_basis_dim = 10
beta = 0.5

In [ ]:
np.random.seed(8675309)
torch.manual_seed(8675309)

num_itr_history, sup_err_history, breaks = lsp_boosting(input_dim, start_basis_dim, end_basis_dim, phi_lam, max_epochs,
                                                        SGD_data_loader, test_data_loader, loss_fn, lam_min, lam_max,
                                                        true_losses, step_size=step_size, const=beta, weighted_avg=True,
                                                        avg_upon_prev=True, thresh_basis=thresh_basis, record_frequency=10,
                                                        distribution='uniform', device=device, trace_frequency=10)

********** now running lsp with #basis dimension = 5 ***********
--------approximate solution path for # itr = 10 complete--------
# epoch: 10	 sup error: 0.11422920227050781
--------approximate solution path for # itr = 20 complete--------
# epoch: 20	 sup error: 0.01976636052131653
--------approximate solution path for # itr = 30 complete--------
# epoch: 30	 sup error: 0.009443342685699463
--------approximate solution path for # itr = 40 complete--------
# epoch: 40	 sup error: 0.002981826663017273
--------approximate solution path for # itr = 50 complete--------
# epoch: 50	 sup error: 0.002051115036010742
--------approximate solution path for # itr = 60 complete--------
# epoch: 60	 sup error: 0.0017272531986236572
--------approximate solution path for # itr = 70 complete--------
# epoch: 70	 sup error: 0.0030344128608703613
--------approximate solution path for # itr = 80 complete--------
# epoch: 80	 sup error: 0.0031889528036117554
--------approximate solution path for # itr = 

In [ ]:
pd.DataFrame(np.array(breaks), columns=['breaks']).to_csv('LSP_results_noisy_boosted_breaks_legendre.csv', index=False)

In [ ]:
file_path = 'LSP_results_noisy_boosted_legendre.csv'

LSP_results_noisy = pd.DataFrame(np.column_stack((num_itr_history, sup_err_history)), columns=['num_itr', 'sup_err'])

# Save the DataFrame to a CSV file
LSP_results_noisy.to_csv(file_path, index=False)

# Read the CSV file into a DataFrame
df = pd.read_csv(file_path)

# Display the DataFrame
df

,num_itr,sup_err
0,10.0,0.114229
1,20.0,0.019766
2,30.0,0.009443
3,40.0,0.002982
4,50.0,0.002051
...,...,...
88,890.0,0.000264
89,900.0,0.000271
90,910.0,0.000286
91,920.0,0.000291
